In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import os

In [ ]:
host=os.environ['host']
database=os.environ['database']
user=os.environ['user']
password=os.environ['password']
port=os.environ['port']

equity_data = pd.read_csv(r'../../data/equity_value_data.csv', parse_dates=['timestamp'])
features_data =  pd.read_csv(r'../../data/features_data.csv')

features_data['time_spent'] = pd.to_datetime(features_data['time_spent'])
features_data.dtypes

equity_data['timestamp'] = pd.to_datetime(equity_data['timestamp'])
equity_data.dtypes

equity_data['timestamp'] = equity_data['timestamp'].dt.tz_localize(None)
equity_data.dtypes

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

features_data.to_sql('features_data', engine, if_exists='replace', index=False)
equity_data.to_sql('equity_value_data', engine, if_exists='replace', index=False)

try:
    with engine.connect() as conn:
        conn.execute("COMMIT")  # End transaction block
        conn.execute('CREATE DATABASE "robinhood";')
    print("Database 'robinhood' created successfully.")
except Exception as e:
    print(f"Error creating database: {e}")